In [1]:
from abc import ABC, abstractmethod
from stable_baselines3 import A2C
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3 import A2C as sb_A2C
from gym import spaces
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import yaml
import gym
from datetime import datetime
from sys import path
from os.path import dirname as dir

path.append(dir(path[0]))
print(path)
#__package__ = "examples"

"""
from FinancialEnvLayer.datacollector import CustomDatasetImporter
from FinancialEnvLayer.datacollector import DataDownloader
from FinancialEnvLayer.dataprocessor import FeatureEngineer
"""

from FinancialDataLayer.DataCollection.DataDownloader import DataDownloader
from FinancialDataLayer.DataProcessing.DefaultFeatureEngineer import DefaultFeatureEngineer

['/Users/doganparlak/Desktop/Master_2.2/Master_Project/uniFi_github/uniFi/AgentLayer', '/usr/local/Cellar/python@3.8/3.8.8_1/Frameworks/Python.framework/Versions/3.8/lib/python38.zip', '/usr/local/Cellar/python@3.8/3.8.8_1/Frameworks/Python.framework/Versions/3.8/lib/python3.8', '/usr/local/Cellar/python@3.8/3.8.8_1/Frameworks/Python.framework/Versions/3.8/lib/python3.8/lib-dynload', '', '/usr/local/lib/python3.8/site-packages', '/usr/local/lib/python3.8/site-packages/selenium-3.141.0-py3.8.egg', '/usr/local/lib/python3.8/site-packages/urllib3-1.26.4-py3.8.egg', '/Users/doganparlak/Desktop/Master_2.2/Master_Project/uniFi_github/uniFi']


In [2]:
class Environment(gym.Env, ABC):

    @abstractmethod
    def reset(self):
        pass

    @abstractmethod
    def step(self, action):
        pass

    @abstractmethod
    def render(self, mode="human"):
        pass

    @abstractmethod
    def get_env(self):
        pass

    @staticmethod
    def softmax_normalization(actions):
        numerator = np.exp(actions)
        denominator = np.sum(np.exp(actions))
        softmax_output = numerator / denominator
        return softmax_output


In [3]:
class Agent(ABC):

    @abstractmethod
    def train_model():
        pass

    @abstractmethod
    def predict():
        pass

    @abstractmethod
    def save_model():
        pass
    
    @abstractmethod
    def load_model():
        pass

In [4]:
class ConventionalAgent(Agent, ABC):

    @abstractmethod
    def train_model():
        pass

    @abstractmethod
    def predict():
        pass

    @abstractmethod
    def save_model():
        pass

    @abstractmethod
    def load_model():
        pass
    
    @abstractmethod
    def _return_predict():
        pass

    @abstractmethod
    def _weight_optimization():
        pass

In [5]:
class RLAgent(Agent, ABC):

    @abstractmethod
    def train_model():
        pass

    @abstractmethod
    def predict():
        
        pass

    @abstractmethod
    def save_model():
        pass

    @abstractmethod
    def load_model():
        pass

In [18]:
class PortfolioEnv(Environment):

    def __init__(self,
                 df: pd.DataFrame,  # input data
                 stock_dim: int,  # number of unique securities in the investment universe
                 hmax: float,  # maximum number of shares to trade
                 initial_amount: float,  # initial cash value
                 transaction_cost_pct: float,  # transaction cost percentage per trade
                 reward_scaling: float,  # scaling factor for reward as training progresses
                 state_space: int,  # the dimension of input features (state space)
                 action_space: int,  # number of actions, which is equal to portfolio dimension
                 tech_indicator_list: list,  # a list of technical indicator names
                 turbulence_threshold=None,  # a threshold to control risk aversion
                 lookback=252,  #
                 day=0):  # an increment number to control date

        self.df = df
        self.day = day
        self.lookback = lookback
        self.stock_dim = stock_dim
        self.hmax = hmax
        self.initial_amount = initial_amount
        self.transaction_cost_pct = transaction_cost_pct
        self.reward_scaling = reward_scaling
        self.state_space = state_space
        self.action_space = action_space
        self.tech_indicator_list = tech_indicator_list

        # action_space normalization and shape is self.stock_dim
        self.action_space = spaces.Box(low=0, high=1, shape=(self.action_space,))
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf,
                                            shape=(self.state_space + len(self.tech_indicator_list), self.state_space))
        

        # load data from a pandas dataframe
        
        ##FINRL APPROACH
        #self.df.set_index("date", drop = True, inplace=True)
        
        self.data = self.df.loc[self.day,:]
        self.covs = self.data['cov_list'].values[0]
        self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
        self.terminal = False
        #self.turbulence_threshold = turbulence_threshold
        # initalize state: initial portfolio return + individual stock return + individual weights
        self.portfolio_value = self.initial_amount

        # memorize portfolio value each step
        self.asset_memory = [self.initial_amount]
        # memorize portfolio return each step
        self.portfolio_return_memory = [0]
        self.actions_memory = [[1 / self.stock_dim] * self.stock_dim]
        self.date_memory=[self.data.date.unique()[0]]

    def reset(self):
        self.asset_memory = [self.initial_amount]
        self.day = 0
        self.data = self.df.loc[self.day,:]
        # load states
        self.covs = self.data['cov_list'].values[0]
        self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
                
        self.portfolio_value = self.initial_amount
        #self.cost = 0
        #self.trades = 0
        self.terminal = False 
        self.portfolio_return_memory = [0]
        self.actions_memory=[[1/self.stock_dim] * self.stock_dim]
        self.date_memory=[self.data.date.unique()[0]] 
        return self.state

    def step(self, actions):
        self.terminal = self.day >= len(self.df.index.unique()) - 1
        if self.terminal:
            df = pd.DataFrame(self.portfolio_return_memory)
            df.columns = ['daily_return']
            plt.plot(df.daily_return.cumsum(), 'r')
            plt.savefig('results/cumulative_reward.png')
            plt.close()

            plt.plot(self.portfolio_return_memory, 'r')
            plt.savefig('results/rewards.png')
            plt.close()

            print("=================================")
            print("begin_total_asset:{}".format(self.asset_memory[0]))
            print("end_total_asset:{}".format(self.portfolio_value))

            df_daily_return = pd.DataFrame(self.portfolio_return_memory)
            df_daily_return.columns = ['daily_return']
            if df_daily_return['daily_return'].std() != 0:
                sharpe = (252 ** 0.5) * df_daily_return['daily_return'].mean() / \
                         df_daily_return['daily_return'].std()
                print("Sharpe: ", sharpe)
            print("=================================")

            return self.state, self.reward, self.terminal, {}

        else:
            weights = Environment.softmax_normalization(actions)
            self.actions_memory.append(weights)
            last_day_memory = self.data

            # load next state
            self.day += 1
            self.data = self.df.loc[self.day, :]
            self.covs = self.data['cov_list'].values[0]
            self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)         
            portfolio_return = sum(((self.data.close.values / last_day_memory.close.values) - 1) * weights)
            log_portfolio_return = np.log(sum((self.data.close.values / last_day_memory.close.values) * weights))
            # update portfolio value
            new_portfolio_value = self.portfolio_value * (1 + portfolio_return)
            self.portfolio_value = new_portfolio_value

            # save into memory
            self.portfolio_return_memory.append(portfolio_return)
            self.date_memory.append(self.data["date"].unique()[0])
            self.asset_memory.append(new_portfolio_value)

            # the reward is the new portfolio value or end portfolo value
            self.reward = new_portfolio_value

        return self.state, self.reward, self.terminal, {}

    def render(self, mode='human'):
        return self.state

    def save_asset_memory(self):
        date_list = self.date_memory
        portfolio_return = self.portfolio_return_memory
        # print(len(date_list))
        # print(len(asset_list))
        df_account_value = pd.DataFrame({'date': date_list, 'daily_return': portfolio_return})
        return df_account_value

    def save_action_memory(self):
        # date and close price length must match actions length
        date_list = self.date_memory
        df_date = pd.DataFrame(date_list)
        df_date.columns = ['date']

        action_list = self.actions_memory
        df_actions = pd.DataFrame(action_list)
        df_actions.columns = self.data.tic.values
        df_actions.index = df_date.date
        # df_actions = pd.DataFrame({'date':date_list,'actions':action_list})
        return df_actions

    def _seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

    def get_env(self):
        e = DummyVecEnv([lambda: self])
        obs = e.reset()
        return e, obs

In [7]:
class A2C(RLAgent):

    def __init__(self,
                 policy= "MlpPolicy",
                 env= None,
                 learning_rate: float = 7e-4,
                 n_steps: int = 5,
                 gamma: float = 0.99,
                 gae_lambda: float = 1.0,
                 ent_coef: float = 0.0,
                 vf_coef: float = 0.5,
                 max_grad_norm: float = 0.5,
                 rms_prop_eps: float = 1e-5,
                 use_rms_prop: bool = True,
                 use_sde: bool = False,
                 sde_sample_freq: int = -1,
                 normalize_advantage: bool = False,
                 tensorboard_log=None,
                 create_eval_env: bool = False,
                 policy_kwargs=None,
                 verbose: int = 0,
                 seed=None,
                 device="auto",
                 _init_setup_model: bool = True):

        self.env = env
        # self.model = A2C(model_params["policy"], model_params["environment"], model_params["verbose"])
        self.model = sb_A2C(policy = policy,
                            env=self.env,
                            learning_rate = learning_rate,
                            n_steps = n_steps,
                            gamma = gamma,
                            gae_lambda= gae_lambda,
                            ent_coef = ent_coef,
                            vf_coef = vf_coef,
                            max_grad_norm = max_grad_norm,
                            rms_prop_eps= rms_prop_eps,
                            use_rms_prop= use_rms_prop,
                            use_sde= use_sde,
                            sde_sample_freq= sde_sample_freq,
                            normalize_advantage= normalize_advantage,
                            tensorboard_log=tensorboard_log,  
                            create_eval_env= create_eval_env,
                            policy_kwargs=policy_kwargs,
                            verbose=verbose,
                            seed=seed,
                            device= device,
                            _init_setup_model = _init_setup_model)

    def train_model(self, **train_params):
        self.model = self.model.learn(**train_params)
        return self.model

    def predict(self, environment, **test_params):

        env_test, obs_test = environment.get_env()
        """make a prediction"""
        account_memory = []
        actions_memory = []

        env_test.reset()
        for i in range(len(environment.df.index.unique())):
            action, _states = self.model.predict(obs_test, **test_params)
            obs_test, rewards, dones, info = env_test.step(action)
            if i == (len(environment.df.index.unique()) - 2):
                account_memory = env_test.env_method(method_name="save_asset_memory")
                actions_memory = env_test.env_method(method_name="save_action_memory")
            if dones[0]:
                print("hit end!")
                break

        return account_memory[0], actions_memory[0]

    def save_model(self, path):
        self.model.save(path)

    def load_model(self, path):
        self.model = self.model.load(path)
        return self.model

In [8]:
#Gather user parameters
with open("../user_params.yaml", "r") as stream:
    try:
        user_params = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

tickers = user_params["tickers"]
env_kwargs = user_params["env_params"]

In [9]:
print("\nTest 3: Downloading from Yahoo.........")
downloaded_df = DataDownloader(start_date='2009-01-01',
                                end_date='2021-10-31',
                                ticker_list= tickers).download_from_yahoo()
"""
downloaded_df = DataDownloader.download_data(start_date='2009-01-01',
                                                end_date='2021-10-31',
                                             ticker_list=tickers)
"""
print(downloaded_df.head())


Test 3: Downloading from Yahoo.........
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (12924, 8)
        date       open       high        low      close     volume   tic  day
0 2008-12-31   3.070357   3.133571   3.047857   2.606277  607541200  AAPL    2
1 2008-12-31  41.590000  43.049999  41.500000  32.005894    5443100    BA    2
2 2008-12-31  43.700001  45.099998  43.700001  30.628826    6277400   CAT    2
3 2008-12-31  72.900002  74.629997  72.900002  43.670761    9964300   CVX    2
4 2009-01-02   3.067143   3.251429   3.041429   2.771174  746015200  AAPL    4


In [10]:
print("\nTest 4: Feature engineer.........")

df_processed = DefaultFeatureEngineer( use_default= False,
                                       tech_indicator_list= env_kwargs["tech_indicator_list"],
                                       use_vix=True,
                                       use_turbulence=True,
                                       use_covar=True).extend_data(downloaded_df)  # included technical indicators as features

print(df_processed.head())


Test 4: Feature engineer.........
Successfully added technical indicators
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (3231, 8)
Successfully added vix
Successfully added turbulence index
Successfully added covariances
        date   tic       open       high        low      close       volume  \
0 2009-12-31  AAPL   7.611786   7.619643   7.520000   6.434926  352410800.0   
0 2009-12-31    BA  55.000000  55.220001  54.049999  42.180115    2189400.0   
0 2009-12-31   CAT  57.599998  57.959999  56.990002  40.802937    3859700.0   
0 2009-12-31   CVX  77.720001  77.779999  76.930000  47.191074    4246600.0   
1 2010-01-04  AAPL   7.622500   7.660714   7.585000   6.535085  493729600.0   

   day      macd    boll_ub    boll_lb     rsi_30      cci_30      dx_30  \
0  3.0  0.105229   6.531585   5.633488  60.410574  155.827674  31.312031   
0  3.0  0.448061  43.940992  41.779004  54.118683   17.483296   4.580979   
0  3.0  0.043858  42.300245  40.

In [11]:
env = PortfolioEnv(df=df_processed, **env_kwargs) 

In [12]:
env_train, _ = env.get_env()

----RESET---


In [13]:
env_train.action_space

Box([0. 0. 0. 0.], [1. 1. 1. 1.], (4,), float32)

In [14]:
train_params = user_params["train_params"]
policy_params = user_params["policy_params"]
test_params = user_params["test_params"]

In [15]:
test_params["A2C_PARAMS"]

{'state': None, 'episode_start': None, 'deterministic': True}

In [16]:
#object creation
a2c = A2C(env = env_train, **policy_params["A2C_PARAMS"])

In [17]:
#training
a2c.train_model(**train_params["A2C_PARAMS"])

----RESET---


In [19]:
env_test = PortfolioEnv(df=df_processed, **env_kwargs) 

In [21]:
#predicting
a2c.predict(environment = env_test, **test_params["A2C_PARAMS"])

begin_total_asset:1000000
end_total_asset:9607008.914902156
Sharpe:  0.9433592168585526
hit end!


(           date  daily_return
 0    2009-12-31      0.000000
 1    2010-01-04      0.026010
 2    2010-01-05      0.011726
 3    2010-01-06      0.000817
 4    2010-01-07      0.008599
 ...         ...           ...
 2973 2021-10-22     -0.002201
 2974 2021-10-25      0.003182
 2975 2021-10-26     -0.004476
 2976 2021-10-27     -0.011978
 2977 2021-10-28      0.020703
 
 [2978 rows x 2 columns],
                 AAPL        BA       CAT       CVX
 date                                              
 2009-12-31  0.250000  0.250000  0.250000  0.250000
 2010-01-04  0.301896  0.221011  0.232432  0.244662
 2010-01-05  0.402806  0.221779  0.227232  0.148184
 2010-01-06  0.403058  0.214172  0.234493  0.148277
 2010-01-07  0.407215  0.223018  0.219962  0.149806
 ...              ...       ...       ...       ...
 2021-10-22  0.350157  0.248862  0.225345  0.175635
 2021-10-25  0.345723  0.249985  0.226671  0.177621
 2021-10-26  0.353053  0.253133  0.222549  0.171265
 2021-10-27  0.349636  0.254

In [22]:
#saving 
a2c.save_model("models/a2c_model")

In [23]:
#loading
loaded_a2c_model = a2c.load_model("models/a2c_model")

In [24]:
loaded_a2c_model